# Autproductie

## Overzicht
We kijken naar de dataset `cars.csv`, deze bevat informatie over de verkoop van 2e hands Volvo’s. We hebben hier een aantal vragen over, beantwoord deze vragen door een beetje code te schrijven en de antwoorden op te schrijven in een Jupyter Notebook.

In [4]:
# import necessary libraries
import pandas as pd
import os

In [7]:
# import dataset
df_cars = pd.read_csv(os.path.join('..', 'data', 'input', 'cars.csv'))

In [10]:
print(df_cars.sample(3))

         manufacturer         model  year  mileage  \
138140      Chevrolet     Malibu LT  2021  56036.0   
526817  Mercedes-Benz  GLA 250 Base  2021  29154.0   
9248             Audi     RS 5 2.9T  2021  17541.0   

                            engine                       transmission  \
138140  1.5L I4 16V GDI DOHC Turbo                      Automatic CVT   
526817  2.0L I4 16V GDI DOHC Turbo  8-Speed Automatic with Auto-Shift   
9248                          2.9L                  7-Speed Automatic   

               drivetrain fuel_type    mpg          exterior_color  \
138140  Front-wheel Drive  Gasoline  29-36                   Black   
526817  Front-wheel Drive  Gasoline  25-34  Patagonia Red Metallic   
9248      All-wheel Drive  Gasoline  18-25    Audi Exclusive Color   

         interior_color  accidents_or_damage  one_owner  personal_use_only  \
138140            Black                  0.0        1.0                0.0   
526817  Macchiato Beige                  0.0        1

### Vraag 1:
Wat voor soort data zijn `year`, `fuel_type` en `seller_rating`? Motiveer je antwoord.

In [37]:
# return types
print(df_cars.loc[:,['year', 'fuel_type', 'seller_rating']].dtypes)

# return sample data
print(df_cars.loc[:,['year', 'fuel_type', 'seller_rating']].sample(3))

year               int64
fuel_type         object
seller_rating    float64
price            float64
dtype: object
        year fuel_type  seller_rating
747794  2021  Gasoline            4.6
58714   2018  Gasoline            4.6
618281  2018  Gasoline            NaN


### Vraag 2:
Ik vraag me af hoe een gewoonlijke (mbt prijs) $2^e$ hands Volvo eruit ziet. Kan je deze zoeken en tonen?

In [51]:
# calculate median price
avg_price_volvo = (df_cars[df_cars['manufacturer'] == 'Volvo']
                   ['price'].median()
                  )

# get Volvo's with median price
volvo_avg_price = df_cars[(df_cars['price'] == avg_price_volvo)
                         & (df_cars['manufacturer'] == 'Volvo')]

volvo_avg_price.T

,755471
manufacturer,Volvo
model,XC40 R-Design
year,2020
mileage,35989.0
engine,I-4 cyl
transmission,Automatic with Geartronic
drivetrain,AWD
fuel_type,Gasoline
mpg,NaN
exterior_color,Onyx Black Metallic


### Vraag 3:
We kunnen de verdeling van de prijs van $2^e$ hands Volvo's voorstellen met een histogram:
![alt text](image.png)
 - Waar zou je verwachten dat het gemiddelde +/- ligt voor deze verdeling?
 - Waar zou je verwachten dat de mediaan +/- ligt?

Teken dit histogram met `.hist` en gebruik `.axvline` om de mediaan en het gemiddelde toe te voegen aan de plot. Komt dit overeen met je verwachting?

 - Kan je de emprisiche cummulatieve distributie functie tekenen voor de verdeling die overeenkomt met dit histogram (zie ook `scipy.stats.ecdf`).

### Vraag 4:
Ik wil nu de verschillende fuel types vergelijken hoeveel deze kosten. Hiervoor maak ik volgende afbeelding met meerdere histogrammen hoe kunnen we deze beter maken? Welke alternatieven zijn er nog?
![alt text](image-1.png)

### Vraag 5:
Gegeven volgende boxplots van de prijs van elektrische & benzine wagens, zijn volgende uitspraken waar/onwaar/niet te besluiten & leg uit:

![alt text](image-2.png)

 - De spreiding op de prijs van elektrische wagens is groter dan die van benzinewagens.

 - Een wagen die meer dan 50 000 euro kost heeft een grotere kans om elektrisch te zijn dan benzine.

 - Een elektrische wagen heeft een grotere kans om meer dan 50 000 euro te kosten dan een benzine wagen.

### Vraag 6:
Maak een visualizatie om het verband tussen mileage & price te tonen?

### Vraag 7:
Volgens deze data is de gemiddelde prijs van $2^e$ hands elektrische Volvo’s 48 000 euro; als we veronderstellen dat onze dataset representatief is voor alle 2de hands volvo’s, hoe kan je de accuraatheid van deze schatting nagaan?